In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split,StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score


df_train = pd.read_csv("./train.csv")
df_test = pd.read_csv("./train.csv")
X_train = df_train.drop(columns=['Price'])
y_train = df_train['Price']
print(X_train)
print(y_train)
# X_test= df_test.drop(columns=['Price'])
# y_test= df_test.drop(columns=['Price'])
print(df_train.dtypes)
print(df_train.isnull().sum())

# X_train['Compartments'].fillna(X_train['Compartments'].median(), inplace=True)
# X_train['Weight Capacity'].fillna(X_train['Weight Capacity'].median(), inplace=True)
# def impute_categorical_uniform(df, columns):
#     """
#     Imputes missing values in a categorical column using a uniform distribution.
    
#     Args:
#     df (pd.DataFrame): Input dataframe.
#     column (str): Column name to impute.

#     Returns:
#     pd.DataFrame: Dataframe with imputed values.
#     """
#     df_copy = df.copy()
#     for column in columns:
#         # Find unique categories (excluding NaN)
        
#     # Find unique categories (excluding NaN)
#         unique_categories = df[column].dropna().unique()
    
#     # Count missing values
#         num_missing = df[column].isna().sum()
    
#     # If there are missing values, sample from the unique categories
#         if num_missing > 0:
#             imputed_values = np.random.choice(unique_categories, size=num_missing, replace=True)
        
#         # Assign these values to missing spots
#             df.loc[df[column].isna(), column] = imputed_values

#                 # Sample based on observed category probabilities
#             category_probs = df_copy[column].value_counts(normalize=True)
#             imputed_values = np.random.choice(category_probs.index, size=num_missing, p=category_probs.values)
#             df_copy.loc[df_copy[column].isna(), column] = imputed_values
    
#     return df




# X_train=impute_categorical_uniform(X_train, columns=[['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Style', 'Color', 'Size']], method="probability")


# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)


# encoder = OneHotEncoder(sparse=False, drop='first')
# X_encoded = encoder.fit_transform(X_train[['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']])

# # Convert to a DataFrame
# encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out())

# # Drop original categorical columns from X_train
# X_train = X_train.drop(columns=['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Style', 'Color'])

# # Merge the transformed data back
# X_train = pd.concat([X_train, encoded_df], axis=1)
# encoder2 = LabelEncoder()
# X_train['Size'] = encoder2.fit_transform(X_train['Size'])

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)







# # Sample dataset (Replace with your actual dataset)

# # Define Stratified K-Fold


# def build_model(input_shape):
#     model = keras.Sequential([
#         keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
#         keras.layers.Dense(32, activation='relu'),
#         keras.layers.Dense(1, activation='sigmoid')  # Binary classification
#     ])
#     model.compile(optimizer='adam', loss='mse', metrics=[keras.metrics.RootMeanSquaredError()])
#     return model




# keras_estimator = KerasRegressor(build_fn=build_model, verbose=0)

# param_grid = {
#     'learning_rate': [0.001, 0.01, 0.1],
#     'dropout_rate': [0.2, 0.3, 0.5],
#     'epochs': [50, 100, 200],
#     'batch_size': [32, 64]
# }

# # Define StratifiedKFold
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # Grid Search with StratifiedKFold
# grid = GridSearchCV(estimator=keras_estimator, param_grid=param_grid, cv=cv, scoring='neg_root_mean_squared_error'', n_jobs=-1)
# grid_result = grid.fit(X_train, y_train)


# # Define function to build a model

# # Print validation performance

# # # Training the model
# # # Evaluating the model
# # # Predicting on test data

best_params = grid_result.best_params_
final_model = build_model(input_dim=X_train.shape[1], 
                          learning_rate=best_params['learning_rate'], 
                          dropout_rate=best_params['dropout_rate'])

final_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1, validation_data=(X_test, y_test))

test_loss, test_RMSE = final_model.evaluate(X_test, y_test)
print(f"Final Test Root Mean Squared Error (RMSE): {test_RMSE:.4f}")


# Evaluate the final model on test data
y_test_pred = final_model.predict(X_test)


            id         Brand Material    Size  Compartments  \
0            0      Jansport  Leather  Medium           7.0   
1            1      Jansport   Canvas   Small          10.0   
2            2  Under Armour  Leather   Small           2.0   
3            3          Nike    Nylon   Small           8.0   
4            4        Adidas   Canvas  Medium           1.0   
...        ...           ...      ...     ...           ...   
299995  299995        Adidas  Leather   Small           9.0   
299996  299996      Jansport  Leather   Large           6.0   
299997  299997          Puma   Canvas   Large           9.0   
299998  299998        Adidas    Nylon   Small           1.0   
299999  299999  Under Armour   Canvas   Small           2.0   

       Laptop Compartment Waterproof      Style  Color  Weight Capacity (kg)  
0                     Yes         No       Tote  Black             11.611723  
1                     Yes        Yes  Messenger  Green             27.078537  
2     